#Paquetes necesarios

In [17]:
import cv2  
import math 
import numpy as np
import os
from ultralytics import YOLO
import easyocr




Reducción de resolución manteniendo las proporciones

In [2]:
import cv2

def resize_video(input_video_path, output_video_path, new_width=None, new_height=None):
    # Abre el video original
    cap = cv2.VideoCapture(input_video_path)

    # Verifica si el video se abrió correctamente
    if not cap.isOpened():
        print("Error: No se pudo abrir el video.")
        return

    # Obtiene las dimensiones originales del video
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calcula la nueva resolución manteniendo la proporción
    if new_width is not None:
        scale_ratio = new_width / original_width
        new_height = int(original_height * scale_ratio)
    elif new_height is not None:
        scale_ratio = new_height / original_height
        new_width = int(original_width * scale_ratio)
    else:
        print("Error: Debes especificar un nuevo ancho o un nuevo alto.")
        return

    # Propiedades del video de salida
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Formato del codec

    # Crea un objeto de escritura de video
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (new_width, new_height))

    while True:
        ret, frame = cap.read()
        
        # Si se leyó correctamente el frame
        if ret:
            # Redimensiona el frame
            resized_frame = cv2.resize(frame, (new_width, new_height))
            out.write(resized_frame)
        else:
            break

    # Cierra todos los recursos
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video redimensionado guardado como '{output_video_path}'")

# Uso de la función especificando solo el nuevo ancho o el nuevo alto
resize_video('../../DSC_0047.MOV', '../../resized_720.mp4', new_width=1280)  # Ejemplo con nuevo ancho
# resize_video('ruta_del_video_entrada.mp4', 'ruta_del_video_salida.mp4', new_height=480)  # Ejemplo con nuevo alto

Video redimensionado guardado como '../../resized_720.mp4'


Función para reducir la resolución al máximo aceptable por cuda (hacia abajo, el requisito es que tiene que ser divisible entre 32)

In [3]:
import cv2

def resize_video(input_path, output_path, stride=32):
    # Abre el video original
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error al abrir el video.")
        return

    # Obtiene las dimensiones originales del video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Calcula nuevas dimensiones divisibles por el 'stride'
    new_width = width - (width % stride)
    new_height = height - (height % stride)

    # Obtiene el formato del video y fps
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Crea un objeto de escritura de video con las nuevas dimensiones
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensiona el frame
        resized_frame = cv2.resize(frame, (new_width, new_height))

        # Escribe el frame redimensionado en el nuevo video
        out.write(resized_frame)

    # Libera los recursos
    cap.release()
    out.release()
    print(f"Video redimensionado guardado en {output_path}")

# Uso de la función
input_video_path = '../../resized_720.mp4'  # Ruta al video original
output_video_path = '../../stream.mp4'  # Ruta donde se guardará el video redimensionado
resize_video(input_video_path, output_video_path)


Video redimensionado guardado en ../../stream.mp4


Función para mostrar los FPS de un video

In [4]:
import cv2

# Función para obtener y mostrar los FPS de un video
def print_video_fps(video_path):
    # Abre el video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error al abrir el video.")
        return

    # Obtiene los FPS del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Muestra los FPS
    print(f"FPS del video: {fps}")

    # Libera el objeto de captura
    cap.release()

# Ruta al video
video_path = '../../stream.mp4'  # Reemplaza con la ruta a tu video

# Llama a la función
print_video_fps(video_path)

FPS del video: 59.94


Función para modificar los FPS de un video

In [5]:
import cv2

def convert_video_fps(input_video_path, output_video_path, target_fps):
    # Abre el video original
    cap = cv2.VideoCapture(input_video_path)

    # Verifica si el video se abrió correctamente
    if not cap.isOpened():
        print("Error: No se pudo abrir el video.")
        return

    # Obtiene los FPS del video original
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS original: {original_fps}")

    # Calcula el factor de reducción de frames
    frame_reduction_factor = int(round(original_fps / target_fps))
    print(f"Factor de reducción de frames: {frame_reduction_factor}")

    # Propiedades del video de salida
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Formato del codec

    # Crea un objeto de escritura de video
    out = cv2.VideoWriter(output_video_path, fourcc, target_fps, (width, height))

    frame_counter = 0

    while True:
        ret, frame = cap.read()
        
        # Si se leyó correctamente el frame
        if ret:
            # Escribe el frame según el factor de reducción
            if frame_counter % frame_reduction_factor == 0:
                out.write(frame)

            frame_counter += 1
        else:
            break

    # Cierra todos los recursos
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video convertido guardado como '{output_video_path}'")

# Uso de la función
convert_video_fps('../../stream.mp4', '../../resized_720_24_fps.mp4', 24)


FPS original: 59.94
Factor de reducción de frames: 2
Video convertido guardado como '../../resized_720_24_fps.mp4'


Modificación de bitrate

In [2]:
import subprocess

def make_video_lighter(input_video_path, output_video_path, bitrate='1M'):
    """
    Reduce el tamaño de un video ajustando su bitrate.
    
    Args:
    input_video_path (str): Ruta al video de entrada.
    output_video_path (str): Ruta donde se guardará el video de salida.
    bitrate (str): Bitrate deseado para el video de salida (por ejemplo, '1M' para 1 Mbps).
    """
    try:
        # Comando FFmpeg para ajustar el bitrate
        command = [
            'ffmpeg', '-i', input_video_path, 
            '-b:v', bitrate, 
            '-bufsize', bitrate, 
            output_video_path
        ]

        # Ejecutar el comando
        subprocess.run(command, check=True)
        print(f"Video reducido guardado como '{output_video_path}' con bitrate de {bitrate}.")
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar FFmpeg: {e}")

# Uso de la función
make_video_lighter('../../resized_720_24_fps.mp4', '../../stream_bitrate2.mp4', '3M') 


Video reducido guardado como '../../stream_bitrate2.mp4' con bitrate de 3M.


Esta ajusta el bitrate y lo deja con el mismo bitrate que el video pasado como primer parámetro

In [1]:
import subprocess
import re

def get_bitrate(video_path):
    """
    Obtiene el bitrate de un video usando FFmpeg.
    """
    command = ['ffmpeg', '-i', video_path]
    result = subprocess.run(command, stderr=subprocess.PIPE, text=True, check=False)
    bitrate_match = re.search(r'bitrate: (\d+) kb/s', result.stderr)
    if bitrate_match:
        print(bitrate_match)
        return bitrate_match.group(1) + 'k'
    else:
        raise ValueError("No se pudo obtener el bitrate del video")

def make_video_lighter(input_video_path, output_video_path):
    """
    Reduce el tamaño de un video manteniendo su bitrate original.
    
    Args:
    input_video_path (str): Ruta al video de entrada.
    output_video_path (str): Ruta donde se guardará el video de salida.
    """
    try:
        # Obtiene el bitrate del video de entrada
        bitrate = get_bitrate(input_video_path)

        # Comando FFmpeg para ajustar el bitrate
        command = [
            'ffmpeg', '-i', input_video_path, 
            '-b:v', bitrate, 
            '-bufsize', bitrate, 
            output_video_path
        ]

        # Ejecutar el comando
        subprocess.run(command, check=True)
        print(f"Video procesado guardado como '{output_video_path}' con bitrate de {bitrate}.")
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar FFmpeg: {e}")
    except ValueError as e:
        print(e)

# Uso de la función
make_video_lighter('../../stream.mp4', '../../prueba2.mp4')


<re.Match object; span=(1739, 1757), match='bitrate: 8579 kb/s'>


In [1]:
import cv2
from ultralytics import YOLO  # Asegúrate de que la biblioteca de YOLO esté instalada
import torch
import math

# Carga del modelo YOLO y uso de la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO('yolov8n.pt').to(device)

print(device)

# Nombre de las distintas clases (filtrado a vehículos y personas)
classNames = ["person", "bicycle", "car", "motorbike", "bus", "train", "truck"]

# Captura desde un archivo de video
vid = cv2.VideoCapture('../../stream_bitrate2.mp4')

while(True):
    ret, img = vid.read()
  
    if ret:
        # Preparar la imagen para el modelo YOLO
        img_tensor = torch.from_numpy(img).to(device).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        results = model(img_tensor, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                cls = int(box.cls[0])
                
                # Verificar si el índice cls está dentro del rango de classNames
                if 0 <= cls < len(classNames):
                    class_name = classNames[cls]

                    # Determinar el color del contenedor basado en la clase
                    if class_name == "person":
                        color = (255, 0, 0)  # Azul para personas
                    else:
                        color = (0, 0, 255)  # Rojo para vehículos

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    
                    confidence = math.ceil((box.conf[0]*100))/100

                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                    cv2.putText(img, class_name, [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        cv2.imshow('Video', img)

    if cv2.waitKey(20) == 27:
        break

vid.release()
cv2.destroyAllWindows()


cuda

0: 704x1280 7 persons, 9 cars, 2 traffic lights, 130.0ms
Speed: 0.0ms preprocess, 130.0ms inference, 72.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 11 persons, 8 cars, 1 traffic light, 1 backpack, 9.5ms
Speed: 0.5ms preprocess, 9.5ms inference, 4.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 8 cars, 1 traffic light, 1 backpack, 3 handbags, 6.0ms
Speed: 0.0ms preprocess, 6.0ms inference, 5.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 6 persons, 7 cars, 2 traffic lights, 1 backpack, 1 handbag, 6.5ms
Speed: 0.0ms preprocess, 6.5ms inference, 2.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 8.0ms
Speed: 0.0ms preprocess, 8.0ms inference, 3.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 7 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 4.0ms postprocess per

Tarea 2:
Comprobamos que tenemos el entorno preparado para usar CUDA

Prueba con zona delimitada

In [3]:
import cv2

# Inicialización de variables globales
punto_inicial = None
punto_final = None
seleccionando = False
zona_seleccionada = False

def seleccionar_zona(event, x, y, flags, param):
    global punto_inicial, punto_final, seleccionando, zona_seleccionada

    if event == cv2.EVENT_LBUTTONDOWN:
        punto_inicial = (x, y)
        seleccionando = True

    elif event == cv2.EVENT_LBUTTONUP:
        punto_final = (x, y)
        seleccionando = False
        zona_seleccionada = True

    elif event == cv2.EVENT_MOUSEMOVE:
        if seleccionando:
            punto_final = (x, y)

def main(video_path):
    global punto_inicial, punto_final, seleccionando, zona_seleccionada

    cap = cv2.VideoCapture(video_path)
    cv2.namedWindow("Video")
    cv2.setMouseCallback("Video", seleccionar_zona)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if punto_inicial and punto_final:
            cv2.rectangle(frame, punto_inicial, punto_final, (0, 255, 0), 2)

        cv2.imshow("Video", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Esc para salir
            break

    cap.release()
    cv2.destroyAllWindows()

    if zona_seleccionada:
        print(f"Zona seleccionada: {punto_inicial} a {punto_final}")

if __name__ == "__main__":
    video_path = '../../stream_bitrate2.mp4'  # Reemplaza con la ruta de tu video
    main(video_path)


Zona seleccionada: (926, 479) a (1269, 677)


(805, 483) a (1101, 618) 
# Definir la zona delimitada (x1, y1, x2, y2)
zona_delimitada = (805, 483, 1101, 618) 

598,314,1132,418

Programa con resaltado

In [15]:
import cv2
from ultralytics import YOLO
import torch
import math

def hay_interseccion(caja1, caja2):
    x1_max = max(caja1[0], caja2[0])
    y1_max = max(caja1[1], caja2[1])
    x2_min = min(caja1[2], caja2[2])
    y2_min = min(caja1[3], caja2[3])
    return x1_max < x2_min and y1_max < y2_min

# Carga del modelo YOLO y uso de la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO('yolov8n.pt').to(device)

print(device)

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "bus", "train", "truck"]

# Captura desde un archivo de video
vid = cv2.VideoCapture('../../stream_bitrate2.mp4')

# Definir la zona delimitada (x1, y1, x2, y2)
zona_delimitada = (2, 336, 289, 435)
zona_delimitada2= (846, 518, 1259, 683)
zona_coches = (1150, 426, 490, -43)

while True:
    ret, img = vid.read()
    contador_personas = 0  # Inicializar el contador de personas en cada frame

    if ret:
        # Preparar la imagen para el modelo YOLO
        img_tensor = torch.from_numpy(img).to(device).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        results = model(img_tensor, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                cls = int(box.cls[0])

                # Índice 0 corresponde a 'person', índice 2 a 'car' (esto puede variar)
                if cls in [0, 2]: 
                    x1, y1, x2, y2 = [int(i) for i in box.xyxy[0]]
                    confidence = math.ceil((box.conf[0]*100))/100

                    caja_objeto = (x1, y1, x2, y2)

                    if hay_interseccion(caja_objeto, zona_delimitada) or hay_interseccion(caja_objeto, zona_delimitada2):
                        color = (0, 255, 0)  # Verde si está dentro de alguna de las zonas
                        if cls == 0:  # Si es persona
                            contador_personas += 1
                    else:
                        if cls == 0:
                            color = (255, 0, 0)  # Azul si está fuera de ambas zonas
                        else:
                            color = (0,0,255)

                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                    etiqueta = classNames[cls] if cls < len(classNames) else 'Unknown'
                    cv2.putText(img, f'{etiqueta}: {confidence}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Dibuja la zona delimitada
        cv2.rectangle(img, zona_delimitada[:2], zona_delimitada[2:], (255, 255, 255), 2)
        cv2.rectangle(img, zona_delimitada2[:2], zona_delimitada2[2:], (255, 255, 255), 2)

        # Mostrar el número de personas en la zona delimitada
        texto_contador = f'Personas en zona: {contador_personas}'
        cv2.putText(img, texto_contador, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Video', img)

        if cv2.waitKey(20) == 27:  # Presiona 'Esc' para salir
            break

vid.release()
cv2.destroyAllWindows()


cuda

0: 704x1280 7 persons, 9 cars, 2 traffic lights, 8.5ms
Speed: 0.0ms preprocess, 8.5ms inference, 3.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 11 persons, 8 cars, 1 traffic light, 1 backpack, 6.5ms
Speed: 0.0ms preprocess, 6.5ms inference, 4.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 8 cars, 1 traffic light, 1 backpack, 3 handbags, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 3.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 6 persons, 7 cars, 2 traffic lights, 1 backpack, 1 handbag, 11.5ms
Speed: 0.0ms preprocess, 11.5ms inference, 2.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 7.5ms
Speed: 0.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 7 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 5.5ms
Speed: 0.0ms preprocess, 5.5ms inference, 4.0ms postprocess per 

In [16]:
import cv2
from ultralytics import YOLO
import torch
import math

def hay_interseccion(caja1, caja2):
    x1_max = max(caja1[0], caja2[0])
    y1_max = max(caja1[1], caja2[1])
    x2_min = min(caja1[2], caja2[2])
    y2_min = min(caja1[3], caja2[3])
    return x1_max < x2_min and y1_max < y2_min

# Carga del modelo YOLO y uso de la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO('yolov8n.pt').to(device)

print(device)

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "bus", "train", "truck"]

# Captura desde un archivo de video
vid = cv2.VideoCapture('../../stream_bitrate2.mp4')

# Definir la zona delimitada (x1, y1, x2, y2)
zona_delimitada = (805, 483, 1101, 618)

while True:
    ret, img = vid.read()
    contador_personas = 0  # Inicializar el contador de personas en cada frame

    if ret:
        # Preparar la imagen para el modelo YOLO
        img_tensor = torch.from_numpy(img).to(device).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        results = model(img_tensor, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                cls = int(box.cls[0])

                # Índice 0 corresponde a 'person', índice 2 a 'car' (esto puede variar)
                if cls in [0, 2]: 
                    x1, y1, x2, y2 = [int(i) for i in box.xyxy[0]]
                    confidence = math.ceil((box.conf[0]*100))/100

                    caja_objeto = (x1, y1, x2, y2)

                    if hay_interseccion(caja_objeto, zona_delimitada):
                        color = (0, 255, 0)  # Verde si está dentro de la zona
                        if cls == 0:  # Si es persona
                            contador_personas += 1
                    else:
                        if cls == 0:
                            color = (255, 0, 0)  # Azul si está fuera de la zona
                        else:
                            color = (0,0,255)

                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                    etiqueta = classNames[cls] if cls < len(classNames) else 'Unknown'
                    cv2.putText(img, f'{etiqueta}: {confidence}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Dibuja la zona delimitada
        cv2.rectangle(img, zona_delimitada[:2], zona_delimitada[2:], (255, 255, 255), 2)

        # Mostrar el número de personas en la zona delimitada
        texto_contador = f'Personas en zona: {contador_personas}'
        cv2.putText(img, texto_contador, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Video', img)

        if cv2.waitKey(20) == 27:  # Presiona 'Esc' para salir
            break

vid.release()
cv2.destroyAllWindows()


cuda

0: 704x1280 7 persons, 9 cars, 2 traffic lights, 5.5ms
Speed: 0.0ms preprocess, 5.5ms inference, 2.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 11 persons, 8 cars, 1 traffic light, 1 backpack, 5.5ms
Speed: 0.0ms preprocess, 5.5ms inference, 4.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 8 cars, 1 traffic light, 1 backpack, 3 handbags, 7.5ms
Speed: 0.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 6 persons, 7 cars, 2 traffic lights, 1 backpack, 1 handbag, 11.0ms
Speed: 0.0ms preprocess, 11.0ms inference, 2.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 8.0ms
Speed: 0.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 7 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 3.0ms postprocess per 

Otro intento, esta vez con vehículos

In [25]:
import cv2
from ultralytics import YOLO
import torch
import math

def hay_interseccion(caja1, caja2):
    x1_max = max(caja1[0], caja2[0])
    y1_max = max(caja1[1], caja2[1])
    x2_min = min(caja1[2], caja2[2])
    y2_min = min(caja1[3], caja2[3])
    return x1_max < x2_min and y1_max < y2_min

# Carga del modelo YOLO y uso de la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO('yolov8n.pt').to(device)

print(device)

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Captura desde un archivo de video
vid = cv2.VideoCapture('../../stream_bitrate2.mp4')

# Definir la zona delimitada (x1, y1, x2, y2)
zona_personas = (805, 483, 1101, 618)
zona_vehiculos = (598,314,1132,418)

while True:
    ret, img = vid.read()
    contador_personas = 0  # Inicializar el contador de personas en cada frame
    contador_vehiculos = 0

    if ret:
        # Preparar la imagen para el modelo YOLO
        img_tensor = torch.from_numpy(img).to(device).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        results = model(img_tensor, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                cls = int(box.cls[0])

                if cls in [0,1,2,3,4,5]: 
                    x1, y1, x2, y2 = [int(i) for i in box.xyxy[0]]
                    confidence = math.ceil((box.conf[0]*100))/100

                    caja_objeto = (x1, y1, x2, y2)

                    if cls == 0 and hay_interseccion(caja_objeto, zona_personas):
                        color = (0, 255, 0)  # Verde si está dentro de la zona
                        if cls == 0:  # Si es persona
                            contador_personas += 1

                    elif cls != 0 and hay_interseccion(caja_objeto, zona_vehiculos):
                        color = (255,255,0)
                        contador_vehiculos +=1

                    else:
                        if cls == 0:
                            color = (255, 0, 0)  # Azul si está fuera de la zona
                        else:
                            color = (0,0,255)
                    """ 
                    if hay_interseccion(caja_objeto, zona_delimitada):
                        color = (0, 255, 0)  # Verde si está dentro de la zona
                        if cls == 0:  # Si es persona
                            contador_personas += 1
                    """
                    
                    
                    
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                    etiqueta = classNames[cls] if cls < len(classNames) else 'Unknown'
                    cv2.putText(img, f'{etiqueta}: {confidence}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Dibuja la zona delimitada
        cv2.rectangle(img, zona_personas[:2], zona_personas[2:], (255, 255, 255), 2)
        cv2.rectangle(img, zona_vehiculos[:2], zona_vehiculos[2:], (255, 255, 255), 2)
        # Mostrar el número de personas en la zona delimitada
        texto_contador = f'Personas en zona: {contador_personas}'
        cv2.putText(img, texto_contador, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Mostrar el número de vehículos en la zona delimitada
        texto_contador = f'Vehiculos en zona: {contador_vehiculos}'
        cv2.putText(img, texto_contador, (25, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Video', img)

        if cv2.waitKey(20) == 27:  # Presiona 'Esc' para salir
            break

vid.release()
cv2.destroyAllWindows()


cuda

0: 704x1280 7 persons, 9 cars, 2 traffic lights, 8.0ms
Speed: 0.0ms preprocess, 8.0ms inference, 2.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 11 persons, 8 cars, 1 traffic light, 1 backpack, 7.5ms
Speed: 0.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 8 cars, 1 traffic light, 1 backpack, 3 handbags, 8.0ms
Speed: 0.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 6 persons, 7 cars, 2 traffic lights, 1 backpack, 1 handbag, 7.5ms
Speed: 0.0ms preprocess, 7.5ms inference, 4.0ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 8 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 6.0ms
Speed: 0.0ms preprocess, 6.0ms inference, 3.5ms postprocess per image at shape (1, 3, 704, 1280)

0: 704x1280 7 persons, 7 cars, 2 traffic lights, 1 backpack, 2 handbags, 6.5ms
Speed: 0.0ms preprocess, 6.5ms inference, 2.0ms postprocess per imag

In [1]:
import cv2
from ultralytics import YOLO  # Asegúrate de que la biblioteca de YOLO esté instalada
import torch
import math

def process_video(input_video_path, output_video_path):
    # Carga del modelo YOLO y uso de la GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = YOLO('yolov8n.pt').to(device)

    print(device)

    # Nombre de las distintas clases (filtrado a vehículos y personas)
    classNames = ["person", "bicycle", "car", "motorbike", "bus", "train", "truck"]

    # Captura desde un archivo de video
    vid = cv2.VideoCapture(input_video_path)

    # Propiedades del video de salida
    fps = vid.get(cv2.CAP_PROP_FPS)
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Formato del codec

    # Crea un objeto de escritura de video
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while(True):
        ret, img = vid.read()
      
        if ret:
            # Preparar la imagen para el modelo YOLO
            img_tensor = torch.from_numpy(img).to(device).permute(2, 0, 1).unsqueeze(0).float() / 255.0
            results = model(img_tensor, stream=True)

            for r in results:
                boxes = r.boxes

                for box in boxes:
                    cls = int(box.cls[0])
                    
                    if 0 <= cls < len(classNames):
                        class_name = classNames[cls]

                        # Determinar el color del contenedor basado en la clase
                        if class_name == "person":
                            color = (255, 0, 0)  # Azul para personas
                        else:
                            color = (0, 0, 255)  # Rojo para vehículos

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        
                        confidence = math.ceil((box.conf[0]*100))/100

                        cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                        cv2.putText(img, class_name, [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            # Escribe el frame procesado en el video de salida
            out.write(img)

        else:
            break

    # Cierra todos los recursos
    vid.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video procesado guardado como '{output_video_path}'")

# Uso de la función
process_video('../../resized.mp4', '../../output.mp4')


cuda

0: 1056x1920 29 persons, 7 cars, 10 airplanes, 2 buss, 1 truck, 1 boat, 1 bench, 2 horses, 136.0ms
Speed: 2.0ms preprocess, 136.0ms inference, 36.3ms postprocess per image at shape (1, 3, 1056, 1920)

0: 1056x1920 26 persons, 7 cars, 10 airplanes, 2 buss, 1 truck, 1 boat, 1 traffic light, 1 bench, 2 horses, 29.4ms
Speed: 0.0ms preprocess, 29.4ms inference, 6.5ms postprocess per image at shape (1, 3, 1056, 1920)

0: 1056x1920 27 persons, 7 cars, 9 airplanes, 2 buss, 1 truck, 1 boat, 1 bench, 2 horses, 29.5ms
Speed: 0.0ms preprocess, 29.5ms inference, 4.0ms postprocess per image at shape (1, 3, 1056, 1920)

0: 1056x1920 27 persons, 8 cars, 10 airplanes, 2 buss, 1 truck, 1 boat, 1 bench, 2 horses, 29.4ms
Speed: 0.0ms preprocess, 29.4ms inference, 8.6ms postprocess per image at shape (1, 3, 1056, 1920)

0: 1056x1920 23 persons, 9 cars, 9 airplanes, 1 bus, 1 boat, 1 bench, 1 horse, 23.5ms
Speed: 0.0ms preprocess, 23.5ms inference, 7.5ms postprocess per image at shape (1, 3, 1056, 1920

In [1]:
import torch
print(torch.cuda.is_available())

True


Imprimimos información de la GPU

In [4]:
!nvidia-smi

Wed Nov 22 11:03:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti   WDDM  | 00000000:2B:00.0  On |                  N/A |
| 30%   45C    P3              78W / 350W |   3091MiB / 12288MiB |      9%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
nw = min([os.cpu_count()])
print(nw)

16


In [1]:

import subprocess
import re
import threading

def monitor_progress(process):
    """
    Monitorea el progreso de FFmpeg y muestra la duración procesada.
    """
    pattern = re.compile(r"time=(\d{2}:\d{2}:\d{2}.\d{2})")
    while True:
        line = process.stderr.readline()
        if not line:
            break
        match = pattern.search(line)
        if match:
            print(f"Tiempo procesado: {match.group(1)}", end='\r')

def make_video_lighter(input_video_path, output_video_path, bitrate='1M'):
    """
    Reduce el tamaño de un video ajustando su bitrate y muestra el progreso.
    
    Args:
    input_video_path (str): Ruta al video de entrada.
    output_video_path (str): Ruta donde se guardará el video de salida.
    bitrate (str): Bitrate deseado para el video de salida (por ejemplo, '1M' para 1 Mbps).
    """
    try:
        # Comando FFmpeg para ajustar el bitrate
        command = [
            'ffmpeg', '-i', input_video_path, 
            '-b:v', bitrate, 
            '-bufsize', bitrate, 
            output_video_path,
            '-progress', 'pipe:2'
        ]

        # Ejecutar el comando
        process = subprocess.Popen(command, stderr=subprocess.PIPE, text=True, bufsize=1)
        
        # Iniciar el monitoreo del progreso en un hilo separado
        progress_thread = threading.Thread(target=monitor_progress, args=(process,))
        progress_thread.start()

        # Esperar a que el proceso termine
        process.wait()
        progress_thread.join()

        print(f"\nVideo reducido guardado como '{output_video_path}' con bitrate de {bitrate}.")
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar FFmpeg: {e}")

# Uso de la función
make_video_lighter('../../stream.mp4', '../../prueba.mp4', '2000k')


In [3]:
!yolo task=detect mode=val model="./runs/detect/train6/weights/best.pt" data="C:/Users/ganma/Downloads/dataset/data.yaml"

Ultralytics YOLOv8.0.215 🚀 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 11, len(boxes) = 204. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                   all        118        204      0.898      0.721      0.802      0.426
Speed: 1.9ms preprocess, 4.6ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\detect\val
💡 Learn more at https://docs.ultralytics.com/modes/val



val: Scanning C:\Users\ganma\Downloads\valid\labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
val: Scanning C:\Users\ganma\Downloads\valid\labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/8 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  12%|█▎        | 1/8 [00:04<00:33,  4.76s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:05<00:13,  2.20s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  38%|███▊      | 3/8 [00:05<00:06,  1.36s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 4/8 [00:05<00:03,  1.08it/s]
                 Class     Images  Instances

In [5]:
!yolo task=detect mode=predict model="./runs/detect/train6/weights/best.pt" conf=0.2 source="./images/" save=True

Ultralytics YOLOv8.0.215 🚀 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\755934778_232384541_1706x960.jpg: 480x800 4 License-Plates, 74.5ms
image 2/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\Hyundai-Tucson-2021-web-1046x616.jpg: 480x800 1 License-Plate, 6.5ms
image 3/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3864.jpg: 800x608 1 License-Plate, 77.5ms
image 4/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3865.jpg: 800x608 2 License-Plates, 7.5ms
image 5/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3866.jpg: 800x608 1 License-Plate, 7.0ms
image 6/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\c3.webp: 608x800 2 License-Plates, 91.0ms
image 7/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\c4.jpg: 544x800 (no detections), 72.5ms
image 8/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\coche1.jpg: 480x800 1 Li

Segundo modelo:

In [5]:
!yolo task=detect mode=predict model="./runs/detect/train4/weights/best.pt" conf=0.4 source="./images/" save=True

Ultralytics YOLOv8.0.215 🚀 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\755934778_232384541_1706x960.jpg: 256x416 4 License_Plates, 78.0ms
image 2/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\Hyundai-Tucson-2021-web-1046x616.jpg: 256x416 1 License_Plate, 5.5ms
image 3/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3864.jpg: 416x320 1 License_Plate, 92.5ms
image 4/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3865.jpg: 416x320 1 License_Plate, 7.0ms
image 5/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3866.jpg: 416x320 1 License_Plate, 6.0ms
image 6/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\c3.webp: 320x416 1 License_Plate, 75.0ms
image 7/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\c4.jpg: 288x416 2 License_Plates, 70.0ms
image 8/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\coche1.jpg: 256x416 1 Lic

In [19]:
# Carga del modelo
model = YOLO('yolov8n.pt')
customModel = YOLO("./runs/detect/train4/weights/best.pt")

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

folder_path = "./images/"

image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

reader = easyocr.Reader(['es'])

def encontrar_matricula_yolo(car_img):
    results = customModel(car_img)
    for r in results:
        boxes = r.boxes

        for box in boxes: 
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(car_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            matricula = car_img[y1:y2, x1:x2]
            # Usar EasyOCR para leer la matrícula
            matricula_resultados = reader.readtext(matricula, allowlist="ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.")
            if matricula_resultados:
                matricula_text = matricula_resultados[0][-2]
                return matricula_text
            
    return "" 

for image_file in image_files:
    # Lee la imagen desde la carpeta
    img = cv2.imread(os.path.join(folder_path, image_file))
    target_size = (800, 600)
    img = cv2.resize(img, target_size)
    
    # Perform inference on an image
    results = model(img)

    # Para cada detección
    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Identificación de la matrícula para coches
            if classNames[int(box.cls[0])] == "car":
                # Dibuja un rectángulo negro alrededor del coche
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 2)

                car = img[y1:y2, x1:x2]
                # Pasar esta mitad inferior a la función encontrar_matricula
                #matricula_text= 
                matricula_text = encontrar_matricula_yolo(car)

                cv2.putText(img, matricula_text, (x1, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                

        # Muestra la imagen con las detecciones
        cv2.imshow('Image', img)
        cv2.waitKey(0)

# Cierra la ventana al finalizar
cv2.destroyAllWindows()


0: 480x640 4 cars, 1 truck, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 320x416 1 License_Plate, 6.0ms
Speed: 0.5ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 License_Plate, 5.5ms
Speed: 0.5ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 416)

0: 352x416 1 License_Plate, 6.0ms
Speed: 0.5ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 416)

0: 320x416 1 License_Plate, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 416)

0: 480x640 1 person, 1 car, 6.5ms
Speed: 1.5ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 288x416 1 License_Plate, 6.0ms
Speed: 0.5ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 416)

0: 480x640 3 cars, 1 truck, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 2.5ms 